In [ ]:
!pip install mlflow boto3 awscli optuna plotly imbalanced-learn lightgbm

In [ ]:
!aws configure set aws_access_key_id #hided for security
!aws configure set aws_secret_access_key #hided for security
!aws configure set default.region us-east-1

In [3]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-3-87-202-243.compute-1.amazonaws.com:5000")

In [4]:
# Set or create an experiment
mlflow.set_experiment("LightGBM HP Tuning")

<Experiment: artifact_location='s3://mlflow-tracking-bucket26/6', creation_time=1772214020954, experiment_id='6', last_update_time=1772214020954, lifecycle_stage='active', name='LightGBM HP Tuning', tags={}>

In [5]:
import pandas as pd

df = pd.read_csv('youtube_preprocessing.csv').dropna()
df.shape

(3118, 3)

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

c:\Users\Urmi Kanrar\anaconda3\envs\sentiment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['sentiment_encoded'] = df['sentiment_encoded'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['sentiment_encoded'])

In [8]:
# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['Comment'])
y = df['sentiment_encoded']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

c:\Users\Urmi Kanrar\anaconda3\envs\sentiment\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Urmi Kanrar\anaconda3\envs\sentiment\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\Urmi Kanrar\anaconda3\envs\sentiment\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Urmi Kanrar\anaconda3\envs\sentiment\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\Urmi Kanrar\anaconda3

In [9]:
# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [10]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy




In [11]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  # L2 regularization

    # Log trial parameters
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
    }

    # Create LightGBM model
    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42)

    # Log each trial as a separate run in MLflow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy




In [12]:
# Step 7: Run Optuna for LightGBM, log the best model, and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=5)  # Increased to 5 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                num_leaves=best_params['num_leaves'],
                                min_child_samples=best_params['min_child_samples'],
                                colsample_bytree=best_params['colsample_bytree'],
                                subsample=best_params['subsample'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=best_params['reg_lambda'],
                                random_state=42)

    # Log the best model with MLflow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [13]:
!pip install nbformat

In [14]:
import nbformat
print(nbformat.__version__)

5.10.4


In [15]:
# Run the experiment for LightGBM
run_optuna_experiment()



[I 2026-02-28 02:35:38,142] A new study created in memory with name: no-name-ca4a3079-54e9-4720-a5cc-865d68a5bece


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003021 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10692
[LightGBM] [Info] Number of data points in the train set: 3794, number of used features: 284
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.099140
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

2026/02/28 02:36:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2026/02/28 02:36:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6/runs/36db9b0a272c46cfbd30c20810132eef.
2026/02/28 02:36:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6.
[I 2026-02-28 02:36:19,298] Trial 0 finished with value: 0.6406743940990516 and parameters: {'n_estimators': 119, 'learning_rate': 0.03629069855283366, 'max_depth': 8, 'num_leaves': 56, 'min_child_samples': 43, 'colsample_bytree': 0.9457807480049394, 'subsample': 0.5715833327027591, 'reg_alpha': 0.005538941666522749, 'reg_lambda': 0.00878588780099312}. Best is trial 0 with value: 0.6406743940990516.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14279
[LightGBM] [Info] Number of data points in the train set: 3794, number of used features: 582
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.099140
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

2026/02/28 02:36:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2026/02/28 02:37:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial_1_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6/runs/f92a902af4fc44d79a382f8b78e16bcd.
2026/02/28 02:37:12 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6.
[I 2026-02-28 02:37:12,903] Trial 1 finished with value: 0.631190727081138 and parameters: {'n_estimators': 234, 'learning_rate': 0.00022118127739889924, 'max_depth': 13, 'num_leaves': 91, 'min_child_samples': 25, 'colsample_bytree': 0.6093621061298974, 'subsample': 0.5349683720406505, 'reg_alpha': 0.0005188312172445186, 'reg_lambda': 0.013170651292043232}. Best is trial 0 with value: 0.6406743940990516.


...

2026/02/28 02:38:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2026/02/28 02:39:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial_3_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6/runs/97485eb44b094b89b3a7dead52c13bd9.
2026/02/28 02:39:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6.
[I 2026-02-28 02:39:02,517] Trial 3 finished with value: 0.59536354056902 and parameters: {'n_estimators': 964, 'learning_rate': 0.03382294939648281, 'max_depth': 8, 'num_leaves': 109, 'min_child_samples': 94, 'colsample_bytree': 0.8605897292671906, 'subsample': 0.5085976362317716, 'reg_alpha': 0.06146287337122097, 'reg_lambda': 0.0009972563492063743}. Best is trial 0 with value: 0.6406743940990516.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003762 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13166
[LightGBM] [Info] Number of data points in the train set: 3794, number of used features: 474
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.099140
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

2026/02/28 02:39:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2026/02/28 02:39:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial_4_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6/runs/625b6ff47207418c9d012ff0235e1c8a.
2026/02/28 02:39:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6.
[I 2026-02-28 02:39:58,933] Trial 4 finished with value: 0.6143308746048473 and parameters: {'n_estimators': 375, 'learning_rate': 0.00021329186686816677, 'max_depth': 10, 'num_leaves': 119, 'min_child_samples': 30, 'colsample_bytree': 0.8916432128871847, 'subsample': 0.5248684916216142, 'reg_alpha': 0.04016239615022011, 'reg_lambda': 0.0034403731065563025}. Best is trial 0 with value: 0.6406743940990516.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002418 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10692
[LightGBM] [Info] Number of data points in the train set: 3794, number of used features: 284
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.098349
[LightGBM] [Info] Start training from score -1.099140
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

2026/02/28 02:40:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2026/02/28 02:40:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run Trial_Best_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6/runs/8260983be73b4d02b2340ac9fd14e1d9.
2026/02/28 02:40:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-3-87-202-243.compute-1.amazonaws.com:5000/#/experiments/6.


In [17]:
best_model

NameError: name 'best_model' is not defined

In [ ]:
import lightgbm
import sklearn

print("LightGBM:", lightgbm.__version__)
print("Scikit-learn:", sklearn.__version__)

LightGBM: 4.5.0
Scikit-learn: 1.8.0


In [ ]:
!pip install plotly